# OGC MovingFeatures Functionality

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

Ressources:

- [OGC Moving Features Standard Working Group on Github](https://github.com/opengeospatial/mf-swg)
- [Moving Features JSON Encoding Standard](https://docs.ogc.org/is/19-045r3/19-045r3.html)
   - [Sample files on Github](https://github.com/opengeospatial/mf-json)

MovingPandas offers OGC MovingFeatures functionality to read and convert MF-JSON files.

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas
import json

from geopandas import GeoDataFrame, read_file
from movingpandas import read_mf_dict, gdf_to_mf_json
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim

import warnings

warnings.filterwarnings("ignore")

opts.defaults(opts.Overlay(active_tools=["wheel_zoom"]))

mpd.show_versions()

## MF-JSON MovingPoint

https://docs.ogc.org/is/19-045r3/19-045r3.html#_mf_json_prism_encoding

https://github.com/opengeospatial/mf-json/tree/master/json-sample/movingpoint

In [ ]:
traj = mpd.read_mf_json("../data/mf-movingpoint.json")
traj.df.head()

In [ ]:
traj.explore(color="red")

## MF-JSON MovingFeatureCollection

https://docs.ogc.org/is/19-045r3/19-045r3.html#_mf_json_prism_encoding

In [ ]:
collection = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {"id": 5},
            "temporalGeometry": {
                "type": "MovingPoint",
                "datetimes": ["2008-02-02T15:02:18Z", "2008-02-02T18:32:28Z"],
                "coordinates": [[116.52299, 40.07757], [116.52302, 39.92129]],
            },
        }
    ],
}

In [ ]:
trajs_collection = read_mf_dict(collection, traj_id_property="id")
trajs_collection

#### Convert TrajectoryCollection to a dictionary compatible with MF-JSON

In [ ]:
df = pd.DataFrame(
    {
        "t": pd.date_range("2020-01-01", periods=5, freq="min"),
        "trajectory_id": [1, 1, 2, 2, 2],
        "geometry": [Point(0, 0), Point(0, 1), Point(1, 2), Point(1, 3), Point(2, 4)],
    }
)
gdf = gpd.GeoDataFrame(df, crs=4326)
tc = mpd.TrajectoryCollection(gdf, traj_id_col="trajectory_id", t="t")
tc

In [ ]:
mf_json = tc.to_mf_json()
mf_json

## MF-JSON Trajectory

https://docs.ogc.org/is/19-045r3/19-045r3.html#_mf_json_trajectory_encoding

https://github.com/opengeospatial/mf-json/tree/master/json-sample/trajectory

In [ ]:
traj = mpd.read_mf_json("../data/mf-trajectory.json", traj_id=3)
traj.df.head()

In [ ]:
traj.explore(color="green")

## Writing MF-JSON

#### Convert Trajectory to a dictionary compatible with MF-JSON

In [ ]:
mf_json = traj.to_mf_json(temporal_columns=["preasure", "wind", "class"])
mf_json

#### Export to JSON file

In [ ]:
with open("../data/mf1.json", "w") as json_file:
    json.dump(mf_json, json_file, indent=4)

#### Read MF-JSON file

In [ ]:
traj = (
    mpd.read_mf_json("../data/mf1.json", traj_id_property="traj_id").trajectories[0].df
)
traj.head()

In [ ]:
traj.explore()

#### Convert GeoDataFrame to a dictionary compatible with MF-JSON

In [ ]:
df = read_file("../data/geolife_small.csv")
gdf = GeoDataFrame(df, geometry=gpd.points_from_xy(df["X"], df["Y"]))
gdf.head()

In [ ]:
mf_json = gdf_to_mf_json(gdf, traj_id_column="trajectory_id", datetime_column="t")

#### Export to JSON file

In [ ]:
with open("../data/mf-geolife_small.json", "w") as json_file:
    json.dump(mf_json, json_file, indent=4)

#### Read JSON file

In [ ]:
with open("../data/mf-geolife_small.json", "r") as file:
    data = json.load(file)

In [ ]:
print(data)

#### Read MF-JSON file from a dictionary

In [ ]:
tc = read_mf_dict(data, traj_id_property="trajectory_id")
tc

#### Read MF-JSON file

In [ ]:
tc = mpd.read_mf_json("../data/mf-geolife_small.json", traj_id_property="trajectory_id")
tc

In [ ]:
tc.explore(column="trajectory_id", cmap="viridis", style_kwds={"weight": 4})